In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.!!!!')


Libraries imported.!!!!


After importing all standard libraries and modules , we import BEatiful Soup for scraping Wikipedia page for Toronto. (as from Turkey we can not reach Wikipedia (dont ask me why) we reach WikiZero and use that url .

In [2]:
from bs4 import BeautifulSoup
print("Beatiful Soup Imported")
!pip install lxml
print("lxml installed ")
!wget -q -O 'html_doc_toronto.html' http://www.wikizero.biz/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N
with open('html_doc_toronto.html') as fp:
    soup = BeautifulSoup(fp)


Beatiful Soup Imported
lxml installed 


We find 'table' by using soup.find 
Th_tags id from finding Table headings  which create Column Names by using get_text function
We define a DataFrame named dfToronto to put all data into.

In [3]:
table = soup.find( "table", {"class":"wikitable sortable"}) 
rows=list()
column_names = [] 
n_columns = 0
n_rows=0
column_names_before_replace=""
column_names_after_replace=""
th_tags = table.find_all('th')
tr_tags = table.find_all('tr')
for th in th_tags:
    tag=str(th.get_text())
    #print(tag,type(tag))
    tag=tag.replace("\n","")
    column_names.append(tag)

#print(column_names)
     
columns = column_names if len(column_names) > 0 else range(0,n_columns)
for row in table.find_all('tr'):
    
     # Determine the number of rows in the table
    td_tags = row.find_all('td')
    
    if len(td_tags) > 0:
        n_rows+=1
        if n_columns == 0:
                        # Set the number of columns for our table
            n_columns = len(td_tags)
    
#print(n_rows)
dfToronto = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
dfToronto_new = pd.DataFrame(columns = columns,
                        index= range(0,n_rows))
#print(dfToronto_new.columns)
print("Toronto DataFrame with Not Assigned Data   shape : ",dfToronto.shape)
#print(dfToronto_new)

Toronto DataFrame with Not Assigned Data   shape :  (288, 3)


For each tr (table record) in the Table 
    Find Td (table data) and put every TD to Toronto DataFrame Columns(use .iat)
    
    
    
    

In [4]:
row_marker=0
column_marker=0
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')   
    for column in columns:
        tagd=column.get_text()
        #print(tagd,type(tagd))
        tagd=tagd.replace("\n","")
        dfToronto.iat[row_marker,column_marker] = tagd
        #print(column.get_text(),tagd,dfToronto.iat[row_marker,column_marker])
        column_marker += 1
        
    if len(columns) > 0:
        row_marker += 1
print("Toronto w/o  Aggregation shape : ",dfToronto.shape)

Toronto w/o  Aggregation shape :  (288, 3)


Cretae a new DataFrame dfToronto_new  with 
    If Neighborhood is 'not Assigned' make Neighborhhod same as Borough
    If Borough is Not Assigned skip that row !

In [5]:
print(dfToronto.shape)
dfToronto_new=dfToronto[dfToronto.Borough != 'Not assigned']
print(dfToronto_new.shape)
dfToronto_new['Neighbourhood'] = np.where(dfToronto_new['Neighbourhood'] == 'Not assigned', 
                                            dfToronto_new['Borough'], dfToronto_new['Neighbourhood'])
print(dfToronto_new.shape)

(288, 3)
(211, 3)
(211, 3)


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


To Aggregate Neighborhoods Let's group by DataFrame PostCode and Borough and use Aggregate function to concanate the Neighborhood data by using lambda function
dfToronto_newer is the final DataFrame

In [6]:
unionofneigh = lambda a: ",".join(a)
dfToronto_newer=dfToronto_new.groupby(['Postcode','Borough'],as_index=False).agg({'Neighbourhood': unionofneigh})
print("Toronto w/o  Aggregation shape : ",dfToronto_new.shape)
print("Toronto with Aggregation shape : ",dfToronto_newer.shape)
dfToronto_newer.rename(columns={"Postcode": "Postal Code"})
dfToronto_newer.columns
dfToronto_newer

Toronto w/o  Aggregation shape :  (211, 3)
Toronto with Aggregation shape :  (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


Now , we have to get Lattitude and Longitude  for each Neighborhood bu using csv and name a new DataFrame

To add longitude and latitude use csv file .


In [7]:
url='http://cocl.us/Geospatial_data' 
dfToronto_LatLong=pd.read_csv(url)
#print(dfToronto_LatLong.shape)
dfToronto_LatLong.rename(index=str,columns={"Postal Code": "Postcode"})
#dfToronto_LatLong.columns

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


We have to merge Dataframe of Toronto with Neighborhood and Dataframe with Longitude/Latitude bu using merge function with left join on the column 'Postcode' and then drop the 'Postal Code' column 

In [9]:
dfToronto_merge=pd.merge(dfToronto_newer,dfToronto_LatLong,left_on='Postcode',
                         right_on='Postal Code',how='left')

#print(dfToronto_merge.columns)
dfToronto_merge.drop('Postal Code',axis=1)
#print(dfToronto_merge.columns)
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfToronto_merge['Borough'].unique()),
        dfToronto_merge.shape[0]
    )
)
dfToronto_merge.shape
dfToronto_new_merge=dfToronto_merge.filter(['Postcode', 'Borough', 'Neighbourhood', 'Latitude',
       'Longitude'],axis=1)
dfToronto_new_merge

The dataframe has 11 boroughs and 103 neighborhoods.


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [ ]:
Insted of using all Borough's we decide to use only Boroughs which includes 'Toronto' ()

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfToronto_new_merge['Borough'].unique()),
        dfToronto_new_merge.shape[0]
    )
)
dfToronto_new_merge=dfToronto_new_merge[dfToronto_new_merge["Borough"]. str.contains("Toronto")]
#print('The dataframe has {} boroughs and {} neighborhoods.'.format(
#        len(dfToronto_new_merge['Borough'].unique()),
#        dfToronto_new_merge.shape[0]
#    )
#)
dfToronto_new_merge

The dataframe has 11 boroughs and 103 neighborhoods.


,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049


Now , as we merged Neighborhoods and Lattitude and LOngitude we can start to draw our Map using Geolocator. 

In [11]:
address = 'Toronto , CA'

geolocator = Nominatim(user_agent="Toronto Explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


We start creating Map of Toronto using Folium with a zoom of 10 . All neighborhoods are in Blu colored  dots with a radius of 5.

In [12]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfToronto_new_merge['Latitude'], dfToronto_new_merge['Longitude'], dfToronto_new_merge['Borough'], dfToronto_new_merge['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [ ]:
Let's use our Client id and Client password to use our Credentials for FourSquare. I have chosen 10 as a limit 

In [13]:
CLIENT_ID = 'T4VGZRZ5ZUOS4VVTFIVFK2A5WX3VCESJCJX0YITATMA4W531' # your Foursquare ID
CLIENT_SECRET = 'BI4VEQEP4R5YRX32W3SEOQVAQ4VOTNSZDFAATHXPDNWJCE4E' # your Foursquare Secret
VERSION = '20190101' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
limit=10

Your credentails:
CLIENT_ID: T4VGZRZ5ZUOS4VVTFIVFK2A5WX3VCESJCJX0YITATMA4W531
CLIENT_SECRET:BI4VEQEP4R5YRX32W3SEOQVAQ4VOTNSZDFAATHXPDNWJCE4E


Just to see our Neigborhoods at Toronto 

In [14]:
#print(type(dfToronto_new_merge))
dfToronto_new_merge.loc[:, 'Neighbourhood']

37                                          The Beaches
41                          The Danforth West,Riverdale
42                        The Beaches West,India Bazaar
43                                      Studio District
44                                        Lawrence Park
45                                     Davisville North
46                                   North Toronto West
47                                           Davisville
48                           Moore Park,Summerhill East
49    Deer Park,Forest Hill SE,Rathnelly,South Hill,...
50                                             Rosedale
51                           Cabbagetown,St. James Town
52                                 Church and Wellesley
53                             Harbourfront,Regent Park
54                              Ryerson,Garden District
55                                       St. James Town
56                                          Berczy Park
57                                   Central Bay

In [15]:
neighborhood_latitude = dfToronto_new_merge.loc[50, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfToronto_new_merge.loc[50, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfToronto_new_merge.loc[50, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [16]:
import requests

radius=500
print(radius)
limit=50
print(limit)
#url = 5
url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,neighborhood_longitude,radius,limit)


500
50


In [64]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cac5acc351e3d6fab546963'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4aff2d47f964a520743522e3',
       'name': 'Rosedale Park',
       'location': {'address': '38 Scholfield Ave.',
        'crossStreet': 'at Edgar Ave.',
        'lat': 43.68232820227814,
        'lng': -79.37893434347683,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68232820227814,
          'lng': -79.37893434347683}],
        'distance': 32

In [65]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        print(categories_list[0])
        return categories_list[0]['name']

In [66]:
venues = results['response']['groups'][0]['items']
#print("RESPONSE -----> ",results['response'])
#print("RESPONSE ve group---->",results['response']['groups'])
#print("RESPONSE ve group ve row 0-----> ",results['response']['groups'][0])
#print("RESPONSE , group , 0 ve items ----->",results['response']['groups'][0]['items'])

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row

print("ONCESI",nearby_venues)
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
print("SONRASI",nearby_venues['venue.categories'])
# clean columns
print(nearby_venues)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

ONCESI              venue.name                                   venue.categories  \
0         Rosedale Park  [{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...   
1          Whitney Park  [{'id': '4bf58dd8d48988d163941735', 'name': 'P...   
2  Alex Murray Parkette  [{'id': '4bf58dd8d48988d163941735', 'name': 'P...   
3        Milkman's Lane  [{'id': '4bf58dd8d48988d159941735', 'name': 'T...   

   venue.location.lat  venue.location.lng  
0           43.682328          -79.378934  
1           43.682036          -79.373788  
2           43.678300          -79.382773  
3           43.676352          -79.373842  
{'id': '4bf58dd8d48988d1e7941735', 'name': 'Playground', 'pluralName': 'Playgrounds', 'shortName': 'Playground', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_', 'suffix': '.png'}, 'primary': True}
{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categ

,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


We define a function getNearbyVenues with an input of (Dataframe , lattitude and longitude and gives 10 nearby venues at that lang/long 500 m radius with a limit of 10 from each neighborhood. The returning Dataframe consists of venues and  lattitude and longitude of eavh venue

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        #rint(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [67]:
#print(dfToronto_new_merge)

toronto_venues = getNearbyVenues(names=dfToronto_new_merge['Neighbourhood'],
                                   latitudes=dfToronto_new_merge['Latitude'],
                                   longitudes=dfToronto_new_merge['Longitude']
                                  )

toronto_venues.head(100)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
5,"The Danforth West,Riverdale",43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
6,"The Danforth West,Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,"The Danforth West,Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
8,"The Danforth West,Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677827,-79.350569,Greek Restaurant
9,"The Danforth West,Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant


In [68]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"Brockton,Exhibition Place,Parkdale Village",20,20,20,20,20,20
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",13,13,13,13,13,13
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,50,50,50,50,50,50
"Chinatown,Grange Park,Kensington Market",50,50,50,50,50,50
Christie,16,16,16,16,16,16


In [69]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 217 uniques categories.


In [79]:
#print(toronto_venues)
print(toronto_venues.columns)
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print(toronto_onehot.columns)
#print(toronto_onehot['Neighborhood'])
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
#print(toronto_onehot['Neighborhood'])
#print(toronto_onehot.shape)
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
print(fixed_columns)
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')
Index(['Adult Boutique', 'Airport', 'Airport Food Court', 'Airport Gate',
       'Airport Lounge', 'Airport Service', 'Airport Terminal',
       'American Restaurant', 'Antique Shop', 'Aquarium',
       ...
       'Thrift / Vintage Store', 'Toy / Game Store', 'Trail', 'Train Station',
       'Vegetarian / Vegan Restaurant', 'Video Game Store',
       'Vietnamese Restaurant', 'Wine Bar', 'Wings Joint', 'Yoga Studio'],
      dtype='object', length=217)
['Yoga Studio', 'Adult Boutique', 'Airport', 'Airport Food Court', 'Airport Gate', 'Airport Lounge', 'Airport Service', 'Airport Terminal', 'American Restaurant', 'Antique Shop', 'Aquarium', 'Argentinian Restaurant', 'Art Gallery', 'Art Museum', 'Arts & Crafts Store', 'Asian Restaurant', 'Athletics & Sports', 'Auto Workshop', 'BBQ Joint', 'Baby Store', 'Bagel Shop', 'Bakery'

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Medical Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [80]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Medical Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.0,0.0,0.0,0.00,0.0,0.0,0.04,0.0,0.000000,0.0,0.0,0.00,0.02,0.0,0.04,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,0.0,0.0,0.02,0.04,0.000000,0.0,0.02,0.020000,0.0,0.0,0.06,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.02,0.06,0.0,0.0,0.0,0.02,0.00,0.000000,0.02,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.02,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.000000,0.000000,0.04,0.0,0.0,0.02,0.0,0.0,0.0,0.02,0.00,0.00,0.02,0.000000,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.00,0.02,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.00,0.0,0.0,0.0,0.0,0.02,0.0,0.02,0.0,0.000000,0.00,0.00,0.0,0.040000,0.000000,0.0,0.02,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.020000,0.0,0.0,0.0,0.0,0.0,0.000000,0.02,0.0,0.00,0.000000,0.0,

In [81]:
toronto_grouped.shape

(38, 217)

In [82]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood]
    #print("TRANSPOSE ONCESI ==== ",temp)
    temp=temp.T.reset_index()
    #print("TRANSPOSE SONRASI ====",temp)
    #print("KOLON  ekleme oncesi",temp)
    #print(temp.columns)
    #print(temp[0])
    temp.columns = ['venue','freq']
    #print("COLONLAR EKLENDI ====",temp)
    temp = temp.iloc[1:]
    #print("ILOC  sonrası ====",temp)
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.06
2           Steakhouse  0.06
3       Breakfast Spot  0.04
4  American Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.06
2    Restaurant  0.04
3    Steakhouse  0.04
4          Café  0.04


----Brockton,Exhibition Place,Parkdale Village----
                venue  freq
0      Breakfast Spot  0.10
1         Coffee Shop  0.10
2                Café  0.10
3  Falafel Restaurant  0.05
4       Burrito Place  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1                 Spa  0.06
2       Garden Center  0.06
3                Park  0.06
4    Recording Studio  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.15
1  Airport Terminal  0.15
2  

In [83]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    print("ROW CATEGORIES AMERICAN RESTA ====",row_categories['American Restaurant'])
    #print("ROW ILOC [0]",row.iloc[0])
    #print("ROW.ILOC [1:] ==== ",row.iloc[1:])
    row_categories_sorted = row_categories.sort_values(ascending=False)
    print("INDEX ====== ",row_categories_sorted.index)
    print("INDEX VALUES ===== ",row_categories_sorted.index.values[0:num_top_venues])
    return row_categories_sorted.index.values[0:num_top_venues]

In [84]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
print(type(indicators))
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
print("TYPE Toronto GROUPED ",type(toronto_grouped))
print("TYPE Toronto GROUPED SHAPE",type(toronto_grouped.shape))
print("SHAPE[0]",toronto_grouped.shape[0])
print("SHAPE[1]",toronto_grouped.shape[1])
for ind in np.arange(toronto_grouped.shape[0]):
    #print("IND =",ind,"  ILOC = ",manhattan_grouped.iloc[ind,:])
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    #print(ind,"  NEIGBORHOOD VENUES SORTED ILOC (IND , 1:)",neighborhoods_venues_sorted.iloc[ind, 1:])
neighborhoods_venues_sorted.head()
#neighborhoods_venues_sorted.shape

<class 'list'>
TYPE Toronto GROUPED  <class 'pandas.core.frame.DataFrame'>
TYPE Toronto GROUPED SHAPE <class 'tuple'>
SHAPE[0] 38
SHAPE[1] 217
ROW CATEGORIES AMERICAN RESTA ==== 0.04
INDEX ======  Index(['Coffee Shop', 'Steakhouse', 'Café', 'Gastropub', 'American Restaurant',
       'Hotel', 'Asian Restaurant', 'Breakfast Spot', 'Bar', 'Pizza Place',
       ...
       'Mexican Restaurant', 'Metro Station', 'Men's Store', 'Medical Center',
       'Martial Arts Dojo', 'Market', 'Malay Restaurant', 'Mac & Cheese Joint',
       'Liquor Store', 'Yoga Studio'],
      dtype='object', length=216)
INDEX VALUES =====  ['Coffee Shop' 'Steakhouse' 'Café' 'Gastropub' 'American Restaurant'
 'Hotel' 'Asian Restaurant' 'Breakfast Spot' 'Bar' 'Pizza Place']
ROW CATEGORIES AMERICAN RESTA ==== 0.0
INDEX ======  Index(['Coffee Shop', 'Cocktail Bar', 'Bakery', 'Restaurant', 'Café',
       'Cheese Shop', 'Farmers Market', 'Seafood Restaurant', 'Steakhouse',
       'Comfort Food Restaurant',
       ...
     

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Café,Gastropub,American Restaurant,Hotel,Asian Restaurant,Breakfast Spot,Bar,Pizza Place
1,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Café,Cheese Shop,Farmers Market,Seafood Restaurant,Steakhouse,Comfort Food Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Gym,Convenience Store,Burrito Place,Stadium,Italian Restaurant,Bar,Caribbean Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Auto Workshop,Skate Park,Smoke Shop,Park,Spa,Brewery,Farmers Market,Fast Food Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Plane,Sculpture Garden


In [85]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
print(kmeans)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)


array([0, 0, 0, 0, 4, 0, 0, 0, 0, 0], dtype=int32)

In [86]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = dfToronto_new_merge

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Health Food Store,Pub,Cupcake Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Yoga Studio,Spa,Caribbean Restaurant,Bakery,Sports Bar,Japanese Restaurant
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Gym,Brewery,Italian Restaurant,Steakhouse,Fish & Chips Shop,Fast Food Restaurant,Liquor Store,Burger Joint,Sushi Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Bank,Bar,Fish Market,Bookstore
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Wings Joint,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run


In [87]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Coffee Shop,Health Food Store,Pub,Cupcake Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Yoga Studio,Spa,Caribbean Restaurant,Bakery,Sports Bar,Japanese Restaurant
42,East Toronto,0,Sandwich Place,Gym,Brewery,Italian Restaurant,Steakhouse,Fish & Chips Shop,Fast Food Restaurant,Liquor Store,Burger Joint,Sushi Restaurant
43,East Toronto,0,Café,Coffee Shop,Bakery,Italian Restaurant,Gastropub,American Restaurant,Bank,Bar,Fish Market,Bookstore
45,Central Toronto,0,Gym,Breakfast Spot,Asian Restaurant,Food & Drink Shop,Burger Joint,Sandwich Place,Hotel,Dance Studio,Clothing Store,Park
46,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Yoga Studio,Fast Food Restaurant,Pet Store,Clothing Store,Gift Shop,Chinese Restaurant,Rental Car Location,Mexican Restaurant
47,Central Toronto,0,Dessert Shop,Pizza Place,Sandwich Place,Coffee Shop,Sushi Restaurant,Restaurant,Italian Restaurant,Café,Park,Brewery
48,Central Toronto,0,Gym,Tennis Court,Restaurant,Playground,Airport Food Court,Coffee Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
49,Central Toronto,0,Coffee Shop,Pub,Pizza Place,American Restaurant,Medical Center,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint
51,Downtown Toronto,0,Coffee Shop,Restaurant,Bakery,Pizza Place,Café,Italian Restaurant,Pub,Butcher,Jewelry Store,Beer Store


In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,1,Garden,Wings Joint,Cupcake Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner


In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,2,Park,Swim School,Bus Line,Wings Joint,Dance Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run


In [91]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,3,Park,Playground,Trail,Cuban Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run,Discount Store,Diner
64,Central Toronto,3,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dog Run


In [92]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,4,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Airport,Airport Food Court,Airport Gate,Harbor / Marina,Plane,Sculpture Garden
